Copyright 2020 DeepMind Technologies Limited

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Clone repo and import dependencies

In [10]:
# import objaverse
# import multiprocessing
# import random
import trimesh
from glob import glob
from tqdm import tqdm
#
# processes = multiprocessing.cpu_count()
# random.seed(42)
# uids = objaverse.load_uids()
# random_object_uids = random.sample(uids, 4)
# objects = objaverse.load_objects(
#     uids=random_object_uids,
#     download_processes=processes
# )

# chair_meshes_paths = glob("D:\\ShapeNetCore.v2\\03001627\\*\\models\\model_normalized.obj")
# table_meshes_path = glob("D:\\ShapeNetCore.v2\\04379243\\*\\models\\model_normalized.obj")




In [11]:
good_meshes = []
for path in tqdm(table_meshes_path):
    with open(path, 'r') as obj_file:
        count=0
        lines = obj_file.readlines()
        for line in lines:
            if line.split(" ")[0]=="v":
                count+=1
    if count < 800:
        good_meshes.append(path)

100%|██████████| 8436/8436 [03:36<00:00, 38.91it/s]


In [12]:
len(good_meshes)

5522

In [14]:
import shutil
for path in tqdm(good_meshes):
    dst = "C:\\Users\\babic\\PycharmProjects\\deepmind-research\\polygen\\data\\tables\\" + path.replace("D:\\ShapeNetCore.v2\\04379243\\", '').replace("\\models\\model_normalized", '')
    shutil.copyfile(path, dst)


100%|██████████| 5522/5522 [00:05<00:00, 1076.16it/s]


In [2]:
# for path in tqdm(chair_meshes_paths):
#     with open(path, 'r') as obj_file:
#         lines = obj_file.readlines()
#         new_lines = []
#         for line in lines:
#             if line.split(" ")[0]=="v":
#                 new_lines.append(line)
#             elif line.split(" ")[0]=="f":
#                 new_lines.append(line)
#         with open(path.replace("model_normalized", "clean"), 'w') as obj_file2:
#             obj_file2.writelines(new_lines)


In [3]:
# chair_meshes_paths_clean = glob("D:\\ShapeNetCore.v2\\03001627\\*\\models\\clean.obj")

In [4]:


import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)  # Hide TF deprecation messages
import matplotlib.pyplot as plt

import modules
import data_utils

In [5]:
ex_list = []
max_verts  = 600 #800
max_faces = 2000 #2800
for path in tqdm(chair_meshes_paths):
    try:
        mesh_dict = data_utils.load_process_mesh(path)
    except IndexError:
        continue
    except ValueError:
        continue
    if(len(mesh_dict['vertices']) > max_verts):
         continue
    if(len(mesh_dict['faces']) > max_faces):
        continue
    mesh_dict['class_label'] = 0 #18
    ex_list.append(mesh_dict)
    # if len(ex_list)>2:
    #     break


 32%|███▏      | 2166/6778 [33:18<1:10:54,  1.08it/s] 


MemoryError: Unable to allocate 50.2 GiB for an array with shape (63483, 849432) and data type bool

In [ ]:
f# def shapenet(paths):
#     for path in paths:
#         # with open(path, 'rb') as obj_file:
#         scene = trimesh.load(path)
#         if not isinstance(scene, trimesh.base.Trimesh):
#             geometries = list(scene.geometry.values())
#             geometry = geometries[0]
#             yield geometry
#         else:
#             yield scene


In [ ]:
# meshes = []
# for path in tqdm(chair_meshes_paths):
#     # with open(path, 'rb') as obj_file:
#     scene = trimesh.load(path)
#     if not isinstance(scene, trimesh.base.Trimesh):
#         geometries = list(scene.geometry.values())
#         geometry = geometries[0]
#         meshes.append(geometry)
#     else:
#         meshes.append(scene)

In [ ]:
# ex_list = []
# max_verts  = 800
# max_faces = 2800
# for mesh in tqdm(meshes):
#     if(mesh.vertices.shape[0] > max_verts):
#          continue
#     try:
#         mesh_dict = data_utils.process_mesh_trimesh(mesh)
#     except ValueError:
#         continue
#     if(len(mesh_dict['vertices']) > max_verts):
#          continue
#     if(len(mesh_dict['faces']) > max_faces):
#         continue
#     mesh_dict['faces']
#       # print(mesh_dict['vertices'])
#     # if(len(mesh_dict['vertices']) > max_verts):
#     #     max_verts = len(mesh_dict['vertices'])
#     # if(len(mesh_dict['faces']) > max_faces):
#     #     max_faces = len(mesh_dict['faces'])
#     mesh_dict['class_label'] = 18
#     ex_list.append(mesh_dict)


In [ ]:
len(ex_list)

In [ ]:
ex_list[1]['faces']

In [6]:
p = ex_list[:2]

In [ ]:
synthetic_dataset = tf.data.Dataset.from_generator(
        lambda: ex_list,
        output_types={
            'vertices': tf.int32, 'faces': tf.int32, 'class_label': tf.int32},
        output_shapes={
            'vertices': tf.TensorShape([None, 3]), 'faces': tf.TensorShape([None]),
            'class_label': tf.TensorShape(())}
        )
# print(max_verts)
# print(max_faces)
ex = synthetic_dataset.make_one_shot_iterator().get_next()

# Inspect the first mesh
with tf.Session() as sess:
  ex_np = sess.run(ex)
print(ex_np)

# Plot the meshes
mesh_list = []
with tf.Session() as sess:
  for i in range(4):
    ex_np = sess.run(ex)
    mesh_list.append(
        {'vertices': data_utils.dequantize_verts(ex_np['vertices']),
         'faces': data_utils.unflatten_faces(ex_np['faces'])})
data_utils.plot_meshes(mesh_list, ax_lims=0.4)

## Vertex model

#### Prepare the dataset for vertex model training
We need to perform some additional processing to make the dataset ready for vertex model training. In particular, `data_utils.make_vertex_model_dataset` flattens the `[V, 3]` vertex arrays, ordering by `Z->Y->X` coordinates. It also creates masks, which are used to mask padded elements in data batches. We also add random shifts to make the modelling task more challenging.

#### Create a vertex model
`modules.VertexModel` is a Sonnet module that. Calling the module on a batch of data will produce outputs which are the sequential predictions for each vertex coordinate. The basis of the vertex model is a Transformer decoder, and we specify it's parameters in `decoder_config`. 



In [ ]:
# Prepare the dataset for vertex model training
vertex_model_dataset = data_utils.make_vertex_model_dataset(
    synthetic_dataset, apply_random_shift=False)
vertex_model_dataset = vertex_model_dataset.repeat()
vertex_model_dataset = vertex_model_dataset.padded_batch(
    1, padded_shapes=vertex_model_dataset.output_shapes)
vertex_model_dataset = vertex_model_dataset.prefetch(1)
vertex_model_batch = vertex_model_dataset.make_one_shot_iterator().get_next()

# Create vertex model
vertex_model = modules.VertexModel(
    decoder_config={
        'hidden_size': 128,
        'fc_size': 512, 
        'num_layers': 3,
        'dropout_rate': 0.
    },
    class_conditional=True,
    num_classes=1,
    max_num_input_verts=max_verts ,  # number of vertices in the input mesh, if this is lower than the number of vertices in the mesh, there will be errors during training
    quantization_bits=8,
)
vertex_model_pred_dist = vertex_model(vertex_model_batch)
vertex_model_loss = -tf.reduce_sum(
    vertex_model_pred_dist.log_prob(vertex_model_batch['vertices_flat']) * 
    vertex_model_batch['vertices_flat_mask'])
vertex_samples = vertex_model.sample(
    1, context=vertex_model_batch, max_sample_length=1500, top_p=0.95,
    recenter_verts=False, only_return_complete=False)

print(vertex_model_batch)
print(vertex_model_pred_dist)
print(vertex_samples)

## Face model

#### Prepare the dataset for face model training
We need to perform some additional processing to make the dataset ready for vertex model training. In particular, `data_utils.make_vertex_model_dataset` flattens the `[V, 3]` vertex arrays, ordering by `Z->Y->X` coordinates. It also creates masks, which are used to mask padded elements in data batches. We also add random shifts to make the modelling task more challenging.

#### Create a face model
`modules.VertexModel` is a Sonnet module that. Calling the module on a batch of data will produce outputs which are the sequential predictions for each vertex coordinate. The basis of the vertex model is a Transformer decoder, and we specify it's parameters in `decoder_config`. 

In [ ]:
face_model_dataset = data_utils.make_face_model_dataset(
    synthetic_dataset, apply_random_shift=False)
face_model_dataset = face_model_dataset.repeat()
face_model_dataset = face_model_dataset.padded_batch(
    1, padded_shapes=face_model_dataset.output_shapes)
face_model_dataset = face_model_dataset.prefetch(1)
face_model_batch = face_model_dataset.make_one_shot_iterator().get_next()

# Create face model
face_model = modules.FaceModel(
    encoder_config={
        'hidden_size': 128,
        'fc_size': 512, 
        'num_layers': 3,
        'dropout_rate': 0.
    },
    decoder_config={
        'hidden_size': 128,
        'fc_size': 512, 
        'num_layers': 3,
        'dropout_rate': 0.
    },
    class_conditional=False,
    max_seq_length=max_faces, # number of faces in the input mesh, if this is lower than the number of vertices in the mesh, there will be errors during training
    quantization_bits=8,
    decoder_cross_attention=True,
    use_discrete_vertex_embeddings=True,
)
face_model_pred_dist = face_model(face_model_batch)
face_model_loss = -tf.reduce_sum(
    face_model_pred_dist.log_prob(face_model_batch['faces']) * 
    face_model_batch['faces_mask'])
face_samples = face_model.sample(
    context=vertex_samples, max_sample_length=1500, top_p=0.95,
    only_return_complete=False)
print(face_model_batch)
print(face_model_pred_dist)
print(face_samples)

In [ ]:
# Optimization settings
learning_rate = 5e-4
training_steps = 500*160
check_step = 160

# Create an optimizer an minimize the summed log probability of the mesh 
# sequences
optimizer = tf.train.AdamOptimizer(learning_rate)
vertex_model_optim_op = optimizer.minimize(vertex_model_loss)
face_model_optim_op = optimizer.minimize(face_model_loss)

# Training loop
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for n in range(training_steps):
    if n % check_step == 0:
      v_loss, f_loss = sess.run((vertex_model_loss, face_model_loss))
      print('Step {}'.format(n))
      print('Loss (vertices) {}'.format(v_loss))
      print('Loss (faces) {}'.format(f_loss))
      v_samples_np, f_samples_np, b_np = sess.run(
        (vertex_samples, face_samples, vertex_model_batch))
      mesh_list = []
      for n in range(1):
        mesh_list.append(
            {
                'vertices': v_samples_np['vertices'][n][:v_samples_np['num_vertices'][n]],
                'faces': data_utils.unflatten_faces(
                    f_samples_np['faces'][n][:f_samples_np['num_face_indices'][n]])
            }
        )
      data_utils.plot_meshes(mesh_list, ax_lims=0.5)
    sess.run((vertex_model_optim_op, face_model_optim_op))